<a href="https://colab.research.google.com/github/meirarc/Market_Price/blob/main/Market_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Market Price Web Scraping

## Why Web Scrapping

Price scraping is rising with the growing competition and growth in online marketplaces. A new player in the market can quickly get up to speed using price and content scraping services. This is because when scraper bots attack a website, they potentially gain access to information on Stock Keeping Units (SKU), product listings, historical pricing, and their entire product catalog. This breadth of information can give new rivals a huge competitive edge. Competitors can automate their scraping activity to such an extent that their website automatically reflects the best price upon analyzing prices from competing sites.

 

### However, it´s not possible to scrape all the data with website APIs.
Some websites provide APIs for users to access part of their data. But even though these sites provide APIs, there still exist some data fields that can´t scrape or have no authentication to access.

For example, Amazon provides a Product Advertising API, but the API itself couldn’t provide access to all the information displayed on its product page for people to scrape, like price and others. In this case, the only way to scrape more data, saying the price data field, is to build our scraper by programming or using certain kinds of automated scraper tools.
 
### It is hard to get the correct data, even for programmers.
Sometimes, even if we know how to scrape data on our own by programming, like the example below in Python, we still could not scrape data successfully for various reasons. In most cases, we probably would be forbidden to scrape from certain websites due to our suspicious repeating scraping actions within a short time. 

### Why I started this project
Consumers nowadays are constantly looking for discounts, special offers, and comparing prices in different online businesses. I started this project for personal use and to help me in a very specific scenario where I usually compare the prices between multiple websites and make a decision on where I will buy the products. This was a weekly activity and manual. 

To improve the work and study a little bit more about data manipulation, data analysis, etc, I tested this model to scrap data from the web and help to increase much time on my weekly activity. It would be best if the website is up to date on pricing and if all the markets allow you to scrap the data, but this is not the case and this entire automation is used today for a learning perspective and try to solve many different problems on the way.

## Libraries for data exploration, processing, and web scraping

We need to import a couple of Python libraries that we will need for data exploration, processing, visualization, and web scraping.

- **Pandas** — We will be using pandas data frames for their functions and a tabular representation of our dataset.

- **BeautifulSoup** — For parsing HTML and XML documents. We will be using this library for web scraping.

- **requests**- To download the web page in HMTL format.

- **re** - Work with Regular Expressions

- **json** - Convert string to json obejct

We will be working in the Google Colab for this tutorial, but you are welcome to use a Python IDE of your choice.

Additionally, we are using the **print_function** library and redefined the work for the **print** command to improve the logging and debugging on the Google Colab.


In [ ]:
from __future__ import print_function
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json


## Set some reusable parameters

The following parameters are reusable in the processes. 
- `enable_print` - log the print when this parameter is `True`
- `filename` - import and export the CSV data with the Product and the prices after web scrapping.

In [ ]:
enable_print = False
filename = "/content/drive/MyDrive/Colab/Market_Price.csv"

## Defined the functions

At this point we are working with two types of functions:
- Utility functions -  can be easily copied for other projects without changes based on the use.
- Custom-Specific functions - Necessary to update the code, and logic, and these functions are very specific to solve/get the requirement completed.

### Utilities Functions

#### print functoin
If the parameter `enable_print` is `True` the `print` will be used to log information and help to debug the application.

In [ ]:
def print(*args, **kwargs):
  if enable_print:
    return __builtins__.print(*args, **kwargs)

#### compare_price
this function just compares two prices and provides status about the comparison

In [ ]:
def compare_price(current, new):
  print("(compare_price): current: ",current, " is blank: ", current == "")
  print("(compare_price): new: ", new, " is blank: ", new == "")
  print("(compare_price:) ",current, new)

  current = float(str(current).replace(",",".")) if current != "" and current is not None else 0 
  new = float(str(new).replace(",",".")) if new != "" and new is not None else 0

  status = ""
  status = "Same Price" if new == current else status  
  status = "More Expensive" if new > current else status   
  status = "On Sale" if new < current else status
  return status

### Custom-Specific Functions per Market


#### St. Marche

In [ ]:
def marche(item):
    webpage = requests.get(item["URL"])
    soup = BeautifulSoup(webpage.content, "html.parser")
    info = soup.find(attrs={"class":"name-and-info"}).p.get_text()
    
    product_name = soup.find(attrs={"class":"product-name"}).get_text()
    product_price = float(info.split("R$ ",1)[1].split("/",1)[0].replace(",","."))
    product_unit = info.split("R$ ",1)[1].split("/",1)[1]

    if product_unit.strip() == "kg":
      size = "1"
      price_kg = product_price
    else:
      size = product_name.split(" ")[-1]
      if size[-1] == "g":
        size = float(size.replace("g", ""))/1000
        price_kg = round(product_price/size, 2)
        product_unit = "g"
      else:
        if product_unit.strip() == "un":
          size = "3"
          price_kg = product_price * 3
        else:
          size = ""
          price_kg = ""

    status = compare_price(item["Product Price per Kg"], price_kg)
    
    item["Product Name"] = product_name if product_name is not None else ""
    item["Product Price"] = "{:.2f}".format(product_price) if product_price is not None else ""
    item["Product Unit"] = product_unit if product_unit is not None else ""
    item["Product Size"] = "{:.3f}".format(float(size)) if size is not None else ""
    item["Product Price per Kg"] = "{:.2f}".format(price_kg) if price_kg is not None else ""
    item["Status"] = status if status is not None else ""

    return item

#### Oba
**To-do**
- [ ] Fix the web scrapping for this page - I´m facing some challenges in web scrapping this marketplace. 

In [ ]:
def oba(item):

  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = ""
  product_price = ""
  product_unit = ""
  size = ""
  price_kg = ""

  product_name = soup.find(attrs={"class":re.compile("ProductDetailsStyles")})
  product_name = product_name if product_name is not None else ""
  
  product_price = soup.find(attrs={"class":re.compile("ProductDetailsStyles")})
  product_price = product_price if product_price is not None else ""
  
  product_unit = ""

  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""
  
  return item

#### Hakuo
**To-do**
- [ ] Fix the web scrapping for this page - I´m facing some challenges in web scrapping this marketplace.

In [ ]:
def hakuo(item):
  
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = ""
  product_price = ""
  product_unit = ""
  size = ""
  price_kg = ""

  product_name = item["URL"].split("/")[-1].replace("-", " ")
  product_name = product_name if product_name is not None else ""
  
  product_price = soup.find(attrs={"class":re.compile("area-preco-detalhe-produto")})
  print(product_price)
  product_price = product_price if product_price is not None else ""
  
  product_unit = ""

  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""
  
  return item

#### Sams Club

In [ ]:
def sams(item):
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = soup.find(attrs={"class":re.compile("productBrand")})
  product_name = product_name.get_text().strip() if product_name is not None else "Error Product Name"
  print("(sams) product_name: ",product_name)

  product_price  = soup.find(attrs={"class":re.compile("productShowCasePricePdp")})
  product_price_kg  = soup.find(attrs={"class":re.compile("productShowCasePriceKgPdp")})
  
  price_kg = 0
  price = 0
  unit = ""
  
  if product_price_kg is not None:
    clean_price_kg = product_price_kg.get_text().strip()
    if clean_price_kg != "":
      price_kg = clean_price_kg.split(" ")[-1].replace(u'\xa0', ' ').split("$")[-1].replace(",",".").strip()
      unit = clean_price_kg.split(" ")[0]
  print("(sams) price_kg: ",price_kg)
  print("(sams) unit: ",unit)

  if product_price is not None:
    clean_price = product_price.get_text().strip()
    if clean_price != "":
      price = clean_price.split("$")[-1].replace(",",".").strip()
  print("(sams) price: ",price)

  product_price = price_kg if price_kg != 0 else price
  product_price = product_price if product_price is not None else ""
  print("(sams) product_price: ",product_price)

  product_unit = unit if unit != "" else ""
  product_unit = product_unit if product_unit is not None else ""
  print("(sams) product_unit: ", product_unit)

    
  if product_unit.strip() == "Preço/Kg":
    size = "1"
    price_kg = product_price
    product_unit = "kg"
  else:
    unit = product_name.strip().split(" ")[-1]
    product_unit = unit
    
    if product_unit == "Unidade":
      size = "3"
      price_kg = "{:.2f}".format(float(product_price) * 3) if product_price != "" else ""
      product_unit = "un" 
    else:
      if product_unit[-2:].lower() == "kg": 
        size = product_unit[:-2].replace(",", ".")
        price_kg = round(float(product_price) / float(size), 2)
        product_unit = "kg"
      else:
        print(product_unit)
        if product_unit.strip()[-1].lower() == "g": 
          size = float(product_unit[:-1])/1000
          price_kg = float(product_price) / size if product_price != "" else ""
          product_unit = "g"
        else:
          if product_name.strip() == "Batata Pacote":
              size = "2"
              price_kg = float(product_price) / 2
              product_unit = "kg"
          else:
            if product_name.strip() == "Agrião Verdureira":
              size = "3"
              price_kg = float(product_price) * 2
              product_unit = "un"
            else:
              size = ""
              price_kg = ""
              product_unit = ""

  print("(sams) current, item[Product Price per Kg]: ", item["Product Price per Kg"] == "")
  print("(sams) new, price_kg: ", price_kg)
  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""

  return item

#### Pão de Açucar

In [ ]:
def paodeacucar(item):

  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = soup.find(attrs={"class":"textstyles__TextComponent-w4b5ef-0 iISkjF"}).get_text().strip()
  product_unit = ""
  size = product_name.split(" ")[-1]

  product_price = soup.find(attrs={"class":re.compile("CurrentPrice")})
  product_price = float(product_price.p.get_text().split("R$ ",1)[1].replace(",",".")) if product_price is not None else ""


  if size[-2].lower() == "k":
    size = float(size.lower().replace("kg", "").replace(",","."))
    price_kg = round(product_price/size, 2) if product_price != "" else ""
    product_unit = "kg"
  else: 
    if size[-1] == "g":
      size = float(size.replace("g", ""))
      price_kg = round(product_price/size*1000, 2) if product_price != "" else ""
      product_unit = "g"
    else:
      size = "3"
      product_unit = "un"
      price_kg = product_price * 3

  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""

  return item

## Load and work with CSV data

Now, after the import of the main libraries and defining the functions, the next step is to work with the data. 

To easily store the data somewhere that can be used for offline manipulation in Excel the decision was to use Google Drive to store a CSV.

### Fix the Prices format 
Due to Excel formation for localization currencies, I had to perform some string manipulation before starting to work with the data on JSON objects. Doing this manipulation before creating the JSON object avoids manipulating the data many times under the custom-specifics functions.

In [ ]:
df = pd.read_csv(filename, sep=";", encoding = "utf-8")
df["Product Price per Kg"] = df["Product Price per Kg"].str.replace(",",".")
df["Product Price"] = df["Product Price"].str.replace(",",".")
df["Product"] = df["Product"].str.replace(u'\xa0', u' ')
df = df.fillna("")
df

### Working with JSON objects
After all the manipulations were completed in the previous step we are good to create the JSON objects and easily work with the data. 

**To-do**
- [ ] perform additional data manipulations before creating the JSON objects and remove the data manipulation from the custom-specific functions

In [ ]:
string = df.to_json(orient = 'records')
data = json.loads(string)

### Iterate each record line
For each record, the CVS/JSON checks if it has the URL to perform the request and calls the market specif function to extract the `product name`, `price`, `size`, etc.

In [ ]:
for item in data:
  if item["URL"] is not None and item["URL"] != "":
    marche(item) if item["Market"] == "St. Marche" else None
    sams(item) if item["Market"] == "Sams Club" else None
    paodeacucar(item) if item["Market"] == "Pao de Acucar" else None
    #oba(item) if item["Market"] == "Oba" else None
    #hakuo(item) if item["Market"] == "Hakuo" else None

### Update the Data Frame and Export the Data
The JSON object `data` is getting updated from the market-specific functions with `price`, `size`, etc. At this time we are converting back the JSON object to Pandas DataFrame and updating the original DataFrame with the new data.

In [ ]:
data_df = pd.DataFrame(data)
data_df

In [ ]:
df.update(data_df)
df

#### Export the data for a local Excel data manipulation
By using the same Google Drive integration with the previous CSV file, now we are updating the file with new data. Using the same file we can compare the old prices and set some status for each record.

In [ ]:
df.to_csv(filename, index=False, sep=";", encoding = "utf-8")

## Data classification and visualization

The first data visualization is related to *On Sale* products ordered by *Market*

**To-do**
- [ ] Planning to add more data visualization in future

In [ ]:
df.sort_values(by=["Market"]).query("Status == 'On Sale'")